# Spark 101

In this lesson we will cover the basics of working with spark dataframes, and
show how spark dataframes are different from the pandas dataframes we have
been working with.

While spark dataframes might superficially look like pandas dataframes, and
even share some of the same methods and syntax, it is important to keep in
mind they are 2 seperate types of objects, and, while spark and pandas code
might look superficially similar, it tends to be semantically very different.

We'll begin by creating the spark session:

In [1]:
import pyspark

In [4]:
# create pyspark session:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 13:52:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

## Creating Dataframes

Spark can convert any pandas dataframe into a spark dataframe with a simple
method call. For this lesson, we will use this functionality to demonstrate
the differences between spark and pandas dataframes and explore how to work
with spark dataframes.

In [6]:
import pandas as pd
import numpy as np

np.random.seed(1349)
pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list('abc'), 20))
)
pandas_dataframe.head()

,n,group
0,0,c
1,1,c
2,2,b
3,3,a
4,4,c


Here we start with a simple pandas dataset, and now we will convert it to a
spark dataframe:

In [7]:
# use the spark session itself to create a dataframe
df = spark.createDataFrame(pandas_dataframe)

In [8]:
df

DataFrame[n: bigint, group: string]

Notice that, while we do see the column names, we don't see the data in the
dataframe like we would with a pandas dataframe. This is because spark is
*lazy*, in that it won't show us values until it has to. For the purposes of
looking at the first few rows of our data, we can use the `.show` method.

In [9]:
type(df)

pyspark.sql.dataframe.DataFrame

In [10]:
# df.head()
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    c|
|  1|    c|
|  2|    b|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



Like pandas dataframes, spark dataframes have a .describe method:

In [12]:
df.describe().show()

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



Which, also like pandas, returns another dataframe. However, since this is a
spark dataframe, we have to explicitly show it.

In [13]:
from pydataset import data
mpg = data('mpg')

In [14]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [15]:
mpg = spark.createDataFrame(mpg)

By default spark will show the first 20 rows, but we can specify how many we
want by passing a number to `.show`.

Let's use some different data so that we have a more robust dataset:

In [19]:
mpg.describe().show(2)

23/05/18 14:02:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------+-----+-----------------+------+-----------------+-----+---+------------------+-----------------+----+-----+
|summary|manufacturer|model|            displ|  year|              cyl|trans|drv|               cty|              hwy|  fl|class|
+-------+------------+-----+-----------------+------+-----------------+-----+---+------------------+-----------------+----+-----+
|  count|         234|  234|              234|   234|              234|  234|234|               234|              234| 234|  234|
|   mean|        null| null|3.471794871794872|2003.5|5.888888888888889| null|4.0|16.858974358974358|23.44017094017094|null| null|
+-------+------------+-----+-----------------+------+-----------------+-----+---+------------------+-----------------+----+-----+
only showing top 2 rows



In [20]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

Let's look at another difference from pandas:

In [21]:
# looking at a subset of columns:
# pandas: df[[''model', 'displ', 'cyl']]
# here we use 'select'

mpg.select(mpg.model,mpg.displ,mpg.cyl).show(5)

+-----+-----+---+
|model|displ|cyl|
+-----+-----+---+
|   a4|  1.8|  4|
|   a4|  1.8|  4|
|   a4|  2.0|  4|
|   a4|  2.0|  4|
|   a4|  2.8|  6|
+-----+-----+---+
only showing top 5 rows



In [23]:
mpg.model.show()

TypeError: 'Column' object is not callable

While this expression would produce a Series of values from a pandas
dataframe, for a spark dataframe this produces a Column object, which is an
object that represents a vertical slice of a dataframe, but does not contain
the data itself.

One way to use our column objects is to use them in combination with the
`.select` method. `.select` is very powerful, and lets us specify what data we
want to see in the resulting dataframe.

In [27]:
mpg.select(mpg.hwy,mpg.cty).show(10)

+---+---+
|hwy|cty|
+---+---+
| 29| 18|
| 29| 21|
| 31| 20|
| 30| 21|
| 26| 16|
| 26| 18|
| 27| 18|
| 26| 18|
| 25| 16|
| 28| 20|
+---+---+
only showing top 10 rows



Again, notice that we don't see any data, instead we see the new dataframe
that is produced. To see the actual data, we'll again need to use `.show`

In [ ]:
# .select will give us the set of columns we want to see
# we can reference the columns with dot notation just like pandas Series
# and if we want to see the results of our action, we can chain a .show()
# at the end of the command


Our column objects support a numer of operations, including the arithmetic
operators:

In [28]:
mpg.hwy + 1

Column<'(hwy + 1)'>

In [29]:
mpg.select(mpg.hwy,mpg.hwy+1).show(5)

+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
| 31|       32|
| 30|       31|
| 26|       27|
+---+---------+
only showing top 5 rows



Here we get back a column that represents the values from the original `hwy`
column with 1 added to them. To actually see this data, we'd need to select it
and show the dataframe.

Once we have a column object, we can use the `.alias` method to rename it:

In [32]:
mpg.select(mpg.hwy.alias('highway'),(mpg.hwy +1).alias('highway plus one')).show(5)

+-------+----------------+
|highway|highway plus one|
+-------+----------------+
|     29|              30|
|     29|              30|
|     31|              32|
|     30|              31|
|     26|              27|
+-------+----------------+
only showing top 5 rows



we can also store column objects in variables and reference them

In [ ]:
# col1: the hwy column from the mpg dataframe, aliased as highway_mileage
# col2: the calculation of the highway column divided by two, aliased as highway_mileage_halved


In [33]:
col1 =  mpg.hwy.alias('highway_mileage')
col2 = (mpg.hwy / 2).alias('highway_mileage_halved')

In [34]:
col2

Column<'(hwy / 2) AS highway_mileage_halved'>

In [35]:
mpg.select(col1,col2).show(3)

+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
+---------------+----------------------+
only showing top 3 rows



## Other ways to create columns

In addition to the syntax we've seen above, we can create columns with the
`col` and `expr` functions from `pyspark.sql.functions` module.

### `col`

In [36]:
from pyspark.sql import functions as F

In [38]:
F.col('hwy')

Column<'hwy'>

In [39]:
F.col('hwy').show(3)

TypeError: 'Column' object is not callable

In [41]:
# mpg.select(mpg.hwy).show(5)
mpg.select(F.col('hwy')).show(5)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
| 26|
+---+
only showing top 5 rows



We can mix and match the syntax we use, and the column object produced by the
`col` function is the same as the the previous column object we saw.

In [42]:
avg_column = 2 / (1 / (F.col('hwy')+F.col('cty')))

In [43]:
avg_column

Column<'(2 / (1 / (hwy + cty)))'>

In [59]:
avg_column = (2 / (1 / (F.col('hwy'))+(1/F.col('cty')))).alias('avg_mileage')

In [60]:
avg_column

Column<'(2 / ((1 / hwy) + (1 / cty))) AS avg_mileage'>

In [61]:
mpg.select(avg_column).show(5)

+------------------+
|       avg_mileage|
+------------------+
| 22.21276595744681|
|             24.36|
|24.313725490196077|
| 24.70588235294118|
| 19.80952380952381|
+------------------+
only showing top 5 rows



In [54]:
arith_avg = ((mpg.hwy + mpg.cty) / 2).alias('arith_mean')

In [55]:
arith_avg

Column<'((hwy + cty) / 2) AS arith_mean'>

In [62]:
mpg.select(F.round(avg_column,2).alias('rounded'),arith_avg).show(10)

+-------+----------+
|rounded|arith_mean|
+-------+----------+
|  22.21|      23.5|
|  24.36|      25.0|
|  24.31|      25.5|
|  24.71|      25.5|
|  19.81|      21.0|
|  21.27|      22.0|
|   21.6|      22.5|
|  21.27|      22.0|
|  19.51|      20.5|
|  23.33|      24.0|
+-------+----------+
only showing top 10 rows



In [ ]:
# from mpg,
# select the column hwy, alias it
# select the column cty, alias it,
# select the column avg_column, alias it


Here we create a variable named `avg_column` that represents the average of the
highway and city mileage of each vehicle. This variable is created by using
the `col` function to produce pyspark Column objects and using the arithmetic
operators to combine them.

Next we select the original highway and city mileage columns, in addition to
our new average mileage column. We demonstrate the `col` function to select
the `hwy` column and refer to the city mileage column with the `df.cty`
syntax we saw previously. We also give all of our columns more readable
aliases before showing the resulting dataframe.

### `expr`

The `expr` function is more powerful than `col`. It does everything `col` does
and more. `expr` returns the same type of column object, but allows us to
express manipulations to the column within the string that defines the column.

In [63]:
my_col = F.expr(mpg.hwy+1)

TypeError: Column is not iterable

In [63]:
my_col = F.expr('hwy+1')

TypeError: Column is not iterable

In [64]:
F.col('hwy+1')

Column<'hwy+1'>

In [68]:
mpg.select(F.col('hwy+1')).show(5)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `hwy+1` cannot be resolved. Did you mean one of the following? [`hwy`, `cty`, `cyl`, `year`, `class`].;
'Project ['hwy+1]
+- LogicalRDD [manufacturer#232, model#233, displ#234, year#235L, cyl#236L, trans#237, drv#238, cty#239L, hwy#240L, fl#241, class#242], false


In [67]:
mpg.select(F.expr('hwy+1')).show(5)

+---------+
|(hwy + 1)|
+---------+
|       30|
|       30|
|       32|
|       31|
|       27|
+---------+
only showing top 5 rows



In [73]:
mpg.select(
    F.expr('hwy'),# the same as `col`
    F.expr('hwy+1'),# an arithmetic expression
    F.expr('hwy + 2 AS highway_plus_two')# using an alias
    # a combination of the above
).show(5)

+---+---------+----------------+
|hwy|(hwy + 1)|highway_plus_two|
+---+---------+----------------+
| 29|       30|              31|
| 29|       30|              31|
| 31|       32|              33|
| 30|       31|              32|
| 26|       27|              28|
+---+---------+----------------+
only showing top 5 rows



Note that all the columns created below are identical, and which syntax to use
is merely a style choice.

## Spark SQL

As we've seen through the column definitions, spark is very flexible and
allows us many different ways to express ourselves. Another way that is fairly
different than what we've seen above is through **spark SQL**, which lets us
write SQL queries against our spark dataframes.

In order to start using spark SQL, we'll first "register" the table with
spark:

In [76]:
mpg.createOrReplaceTempView('mpg')

In [77]:
spark.sql(
'''
SELECT cty
FROM mpg



'''
).show(5)

+---+
|cty|
+---+
| 18|
| 21|
| 20|
| 21|
| 16|
+---+
only showing top 5 rows



Now we can write a sql query against the `mpg` table:

In [78]:
spark.sql(
'''
SELECT hwy, (hwy+cty / 2) AS arith_avg
FROM mpg



'''
).show(5)

+---+---------+
|hwy|arith_avg|
+---+---------+
| 29|     38.0|
| 29|     39.5|
| 31|     41.0|
| 30|     40.5|
| 26|     34.0|
+---+---------+
only showing top 5 rows



Notice that the resulting value is another dataframe. As we know, in order to
view the values in a dataframe, we need to use `.show`

It is worth noting that all of the methods for creating / manipulating
dataframes outlined above are the same in terms of performance as well. All of
the resulting dataframes get turned into the same spark code that gets
executed on the JVM, so it really is just a style choice as to which to use.

## Type Casting

We can view the types of the column in our dataframe in one of two ways:

In [ ]:
# pandas: .astype(int)


In [ ]:
# .printSchema() is similar to a .info() call


Both provide the same information.

To convert from one type to another, we can use the `.cast` method on a
column.

Note that if a value is not able to be converted, it will be replaced with
null:

## Basic Built-in Functions

We've used the `col` and `expr` functions, but there are many other functions
within the `pyspark.sql.functions` module, all of which operate on pyspark
dataframe columns. Here we'll demonstrate several:

- `concat`: to concatenate strings
- `sum`: to sum a group
- `avg`: to take the average of a group
- `min`: to find the minimum
- `max`: to find the maximum

**_Note that importing the `sum` function directly will override the built-in
`sum` function._** This means you will get an error if you try to sum a list
of numbers, because `sum` will refernce the pyspark `sum` function, which
works with pyspark dataframe columns, while the built-in `sum` function works
with lists of numbers. The same holds true for the built in `min` and `max`
functions.

In [ ]:
# Note: The pyspark avg and mean functions are aliases of eachother


!!!tip "`pyspark` imports"
    In this lesson we will explicitly import any functions from `pyspark.sql.functions` that we use, but it very common to see something like:
    
    ```
    from pyspark.sql.functions import *
    ```
    
    which will import *all* of the functions from the `pyspark.sql.functions` module.

In [ ]:
# from pyspark.sql.functions import lit

In [ ]:
# we can chain an alias on a concat function call
# we need to cast cylinders with a lit() function call to cast it as a string literal


In order to use a string literal as part of our select, we'll need to use the
`lit` function, otherwise spark will try to resolve our string as a column.

Here we select the concatenation of the number of cylinders (the value from
the `cyl` column) and the string literal " cylinders".

## More `pyspark` Functions for String Manipulation

Let's take a look at a couple more functions for string manipulation.

In [ ]:
# from pyspark.sql.functions import regexp_extract, regexp_replace

In order to demonstrate these functions we'll create a dataframe with some text data.

In [ ]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show()

The `regexp_extract` function lets us specify a regular expression with at least one capture group, and create a new column based on the contents of a capture group.

In [ ]:
# in my select call:
# address, the column as it stands
# regexp_extract, acting as a re.extract, aliasing that with a .alias method 
textdf.select(
    "address",
    regexp_extract(####).alias("street_no"),
    regexp_extract(####).alias("street"),
).show(truncate=False)

In the example above, the first argument to `regexp_extract` is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.

In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression.

In [ ]:
textdf.select(
    "address",
    #regexp_replace('subject', 'pattern', 'replacement)
    regexp_replace(####).alias("city_state_zip"),
).show(truncate=False)

In our example above, we obtain just the city, state, and zip code of the address by replacing everything up to the first comma with an empty string.

## `.filter` and `.where`

Spark provides two dataframe methods, `.filter` and `.where`, which both allow
us to select a subset of the rows of our dataframe.

In [ ]:
# pandas:
# mpg[mpg.cyl == 4]

In [ ]:
# from pyspark.sql.functions import when

## When and Otherwise

Similar to an `IF` in Excel, `CASE...WHEN` in SQL, or `np.where` in python,
spark provides a `when` function.

The `when` function lets us specify a condition, and a value to produce if
that condition is true:

In [ ]:
from pyspark.sql.functions import when

Notice here that if the condition we specified is false, `null` will be
produced. Instead of null, we can specify a value to use if our condition is
false with the `.otherwise` method.

In [ ]:
m

To specify multiple conditions, we can chain `.when` calls. The first
condition that is met will be the value that is used, and if none of the
conditions are met the value specified in the `.otherwise` will be used (or
`null` if you don't provide a `.otherwise`).

Notice here that a car with a `displ` of 1.8 matches both conditions we
specified, but `small` is produced because it is associated with the first
matching condition. For any value between 2 and 3, `medium` will be produced,
and anything larger than 3 will produce `large`.

## Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with
two methods: `.sort`, and `.orderBy`. `.sort` and `.orderBy` are aliases of
each other and do the exact same thing. Like other methods we've seen, `.sort`
takes in a Column object or a string that is the name of a column.

By default, values are sorted in ascending order. To sort in descending order,
we can use the `.desc` method on any Column object, or the `desc` function
from `pyspark.sql.functions`.

In [ ]:
# from pyspark.sql.functions import asc, desc

To specify sorting by multiple columns, we provide each column as a separate
argument to `.sort`.

Here we will first reverse alphabetically by number of cylinders from lowest to highest, then by the vehicle's highway
mileage, from greatest to smallest.

## Grouping and Aggregating

To aggregate our data by group, we can use the `.groupBy` method. Like with
`.select`, we can pass either Column objects or strings that are column names
to `.groupBy`. All of the expressions below are equivalent.

In [ ]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col('cyl'))
mpg.groupBy('cyl')

Once the data is grouped, we need to specify an aggregation. We can use one of
the aggregate functions we imported earlier, alond with a column:

To group by multiple columns, pass each of the columns a a separate argument
to `.groupBy` (Note that this is different from pandas, where we would need to
pass a list).

In addition to `.groupBy`, we can use `.rollup`, which will do the same
aggregations, but will also include the overall total:

Here the null value in `cyl` indicates the total count.

And in the example above, the null row represents the overall average highway
mileage.

## Crosstabs and Pivot Tables

In addition to groupby, spark provides a couple other ways to do aggregation.
One of which is `.crosstab`. This is very similary to pandas `.crosstab`
function, in that it calculates the number of occurances of each unique value
from the two passed columns:

`.crosstab` simply does counts, if we want a different aggregation, we can use
`.pivot`. For example, to find the average highway mileage for each
combination of car class and number of cylinders, we could write the
following:

Here the unique values from the column we group by will be the rows in the
resulting dataframe, and the unique values from the column we pivot on will
become the columns. The values in each cell will be equal to the aggregation
we specified over the group of values defined by the intersection of the rows
and the columns.

## Handling Missing Data

Let's take a look at how spark handles missing data. First we'll create a dataframe that has a few missing values:

In [ ]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

Spark provides two main ways to deal with missing values:

- `.fill`: to replace missing values with a specified value
- `.drop`: to drop rows containing missing values

Both methods are accessed through the `.na` property. We'll look at some examples below:

For both methods, we can specify that we only want to fill or drop values in a specific column with a second argument:

Notice that above the na values in the `x` column were filled with 0, but the na values in y were left alone.

In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

## More Dataframe Manipulation Examples

Let's take a look at some more examples of working with spark dataframes. For
these examples, we'll be working with a dataset of observations of the
weather in seattle.

In [ ]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

Let's print out the number of rows and columns in our dataset:

Let's first find the dates where the data starts and stops:

In [ ]:
min_date, max_date = ####
min_date, max_date

Here we use `.select` to select the minimum date and the maximum date.
`.first` returns us the first row of our results, which consists of two value,
and so can be unpacked into the `min_date` and `max_date` variables.

Next we will combine the temp max and min columns into a single column,
`temp_avg`.

In [ ]:
# withColumn is similar to our df.assign()
weather = ####


Now we will calculate the total amount of rainfall for each month. We'll do
this by first creating a month column, then grouping by the month, and
finally, aggregating by taking the sum of the precipitation. To do this we will need to use the `month` function.

In [ ]:
# from pyspark.sql.functions import month, year, quarter

The `.sort` at the end isn't necessary, but presents that data in a friendlier
way.

Let's now take a look at the average temperature for each type of weather in
December 2013:

In [ ]:
# month will rip out the month numeral from the date
# establish a filter df[df.month == 12]
# pass a second filter of df[df.year == 2013]
# aggregate based on weather tag, present average temperature


Here we first have a couple of `.filter` calls in order to restrict our data
to December of 2013. We then group by the weather column, and lastly,
aggregate by taking the average of our `temp_avg` column. The combination of
group by and agg will calculate the average temperature for each unique value
of the `weather` column.

Let's now find out how many days had freezing temperatures in each month of
2013.

One last example, let's calculate the average temperature for each quarter of
each year:

Here we create the `quarter` and `year` columns, then group by these two new columns, and take the average temperature as our aggregate. Lastly, we sort by the year and quarter for presentation purposes.

We could also use a pivot table like this:

Here instead of grouping by two columns, we grouped by the first column and pivoted on the other column.

## Joins

Like pandas and sql, spark has functionality that lets us combine two tabular
datasets, known as a **join**.

We'll start by creating some data that we can join together:

In [ ]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()

To join two dataframes together, we'll need to call the `.join` method on one
of them and supply the other as an argument. In addition, we'll need to supply
the condition on which we are joining. In our case, we are joining where the
`role_id` column on the users table is equal to the `id` column on the roles
table.

By default, spark will perform an inner join, meaning that records from both
dataframes will have a match with the other. We can also specify either a left
or a right join, which will keep all of the records from either the left or
right side, even if those records don't have a match with the other dataframe.

Notice that examples above have a duplicate `id` column. There are several
ways we could go about dealing with this:

- alias each dataframe + explicitly select columns after joining (this could also be implemented with spark SQL)
- rename duplicated columns before merging
- drop duplicated columns after the merge (`.drop(right.id)`)

## Visualization (or Lack Therof)

Spark does not provide a way to do visualization with their dataframes. To
visualize data from spark, you should use the `.toPandas` method on a spark
dataframe to convert it to a pandas dataframe, then visualize as you normally
would.

!!!warning "Converting to A Pandas Dataframe"
    Converting a spark dataframe to a pandas dataframe will pull all the data into memory, so make sure you have enough available memory to do so.

## Exercises

Using the [repo setup directions](https://ds.codeup.com/fundamentals/git/), setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

1. Load the `mpg` dataset as a spark dataframe.

    1. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

    1. Transform the `trans` column so that it only contains either `manual` or `auto`.

1. Load the `tips` dataset as a spark dataframe.

    1. What percentage of observations are smokers?
    1. Create a column that contains the tip percentage
    1. Calculate the average tip percentage for each combination of sex and smoker.

1. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).